In [13]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import asyncio
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from pydantic import BaseModel
from openai import AsyncOpenAI
 






In [14]:
load_dotenv(override=True)

True

In [15]:
instructions_1  = """Your are a sales agent working for BetaLaunch.\n
It is a company which is oriented at providing software solutions for businesses such as web applications and mobile applications\n
Your write in a professional and serious tone\n
"""

instructions_2 = """Your are a sales agent working for BetaLaunch.\n
It is a company which is oriented at providing software solutions for businesses such as web applications and mobile applications\n
Your write in a engaging and responsive tone\n
"""

instructions_3 = """Your are a sales agent working for BetaLaunch.\n
It is a company which is oriented at providing software solutions for businesses such as web applications and mobile applications\n
Your write in a concise and detailed oriented tone. \n
"""


In [16]:
sales_agent_1 = Agent(name="Professional_sales_agent", instructions=instructions_1,)
sales_agent_2 = Agent(name="Engaging_sales_agent", instructions=instructions_2)
sales_agent_3 = Agent(name="details_oriented_sales_agent", instructions=instructions_3)

In [17]:
gemini_api_key = os.getenv('GEMINI_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

gemini_client =  AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=gemini_api_key)

gemini_model = OpenAIChatCompletionsModel(model='gemini-2.0-flash',openai_client=gemini_client)


In [18]:
message = "Write a cold email"

with trace("Execution of multiple agents"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, message),
        Runner.run(sales_agent_2, message),
        Runner.run(sales_agent_3, message),
    )
    output = [result.final_output for result in results];


In [19]:
evaluation_agent_instructions = f"""
 You act as an evaluation agent, who's job is to evaluate the array of provided emails and pick the best one that a user might reply.\n
 Do not give any explanations, just reply with the selected email\n
"""

In [20]:
evaluation_agent = Agent(name="evaluation agnet", instructions=evaluation_agent_instructions);

evaluation_message =f"The list of emails: \\n {output}"

evaluation_result = await Runner.run(evaluation_agent,evaluation_message )

print(evaluation_result.final_output);




"Subject: Unlock New Potential with Cutting-Edge Software Solutions

Hi [Recipient's Name],

I hope this message finds you well! I'm [Your Name] from BetaLaunch, and I'm excited to introduce you to a world of innovative software solutions tailored to elevate your business.

At BetaLaunch, we specialize in crafting top-notch web and mobile applications, designed to streamline operations and enhance user engagement. Whether you're looking to revamp your current system or build something entirely new, our team is here to bring your vision to life.

Why partner with us?

- **Customized Solutions:** We understand that every business is unique. Our solutions are tailored to meet your specific needs.
- **Expert Team:** With a dedicated team of developers and designers, we ensure high-quality, robust applications.
- **Seamless Integration:** Our applications integrate smoothly with existing systems, minimizing disruption.

I’d love to discuss how we can help your business thrive. Are you avail

In [21]:
@function_tool
def send_email(body: str):
    """Send an email with the given body to a reepient"""
    try:
        sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
        from_email = Email('pharshana719@gmail.com')
        to_email = To('isankadatadownload@gmail.com')
        content =  Content("text/plain", body)
        mail = Mail(from_email, to_email,"Sales Email", content).get()
        response = sg.client.mail.send.post(request_body = mail)
        return {"Status":"Success"}

        
    except Exception as e :
        print(e.message)
    

In [22]:
send_email

FunctionTool(name='send_email', description='Send an email with the given body to a reepient', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000181E2E73C40>, strict_json_schema=True, is_enabled=True)

In [23]:
Description = "Write a cold email"

tool1 = sales_agent_1.as_tool(tool_name = "Write_professional_emails", tool_description= Description)
tool2 = sales_agent_2.as_tool(tool_name = "Write_user_engaging_emails", tool_description=Description)
tool3  = sales_agent_3.as_tool(tool_name = "Write_details_oriented_emails", tool_description=Description)

In [24]:
tool1

FunctionTool(name='Write_professional_emails', description='Write a cold email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Write_professional_emails_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000181E0439300>, strict_json_schema=True, is_enabled=True)

In [25]:
tools = [tool1, tool2,tool3, send_email]

In [26]:
sales_manager_instructions = """Your are are sales manager who's working for BetaLaunch. You use the tools given to generate cold sales emails\n
        You do not generate emails by yourself, instead you always use the tools\n
        You tries all the three types of email generating tools and pickup the best one by comparing the results of those three tools\n 
        The you select the email sending tool which is send_email yo send the best email. Always remember you are only sending the best selected email\n 

 """

sales_manager = Agent(name="Sales Manger", instructions=sales_manager_instructions, tools=tools, model="gpt-4o-mini")


message = "Send a cold sales email addressed to 'Dear CEO'"


with trace("Sales manager"):
    manager_resulst = await Runner.run(sales_manager, message)
    print(manager_resulst)



RunResult:
- Last agent: Agent(name="Sales Manger", ...)
- Final output (str):
    I've successfully sent the cold sales email addressing the CEO. If you need any further assistance or adjustments, feel free to let me know!
- 33 new item(s)
- 7 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [27]:
@function_tool
def send_html_email(subject: str, html_body:str) -> Dict[str, str] :
    """Send out sales emails with a give subet and formatted """
    sg = sendgrid.SendGridAPIClient(api_key= os.environ.get('SENDGRID_API_KEY'))
    from_email = Email('pharshana719@gmail.com')
    to_email = To('isankadatadownload@gmail.com')
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}

    

In [28]:

subject_agent_instructions = """Your are  capable of generating an appropriate subject for a given email message that the user is likely to response"""

html_agent_instructions = """Your are capable of converting a text email body in to a HTML email body\n 
You are given a html body which might have some markdown and you need to convert it to an HTML email body with simple, clear, compelling layout and design.
"""

subject_agent = Agent(name= "Email_Subject_Agent",instructions=subject_agent_instructions)
subject_tool = subject_agent.as_tool(tool_name="subject_writer", tool_description="Write an appropriate subejct for the sales email")

html_agent = Agent(name="HTML_converter", instructions=html_agent_instructions)
html_tool = html_agent.as_tool(tool_name="HTML_converter_tool",tool_description="Convert the text email body in to HTML formatted email body")


In [29]:
tools = [subject_tool, html_tool, send_html_email]

In [30]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

email_agent = Agent(name="Email_sender",
                     instructions=instructions,
                     model="gpt-4o-mini",
                     tools=tools,
                     handoff_description="Convert an email to HTML and send it")

In [31]:
sales_manager_instructions = """You are a sales manager working for BetaLaunch. You use tools given to generate sales emails\n 
You never generate emails by yourself, instead you use tools. You try all the three email generating tools and pick up the best response among them\n 
You can use a tool more than once if a tool did not provide and appropriate email body. Then you select the best email body among them that a user will
most likeyly to reply to.\n After selecting the best email body you handoff to the Email Manager Agent to format and send the email. 

"""

new_tools_list = [tool1, tool2, tool3]
handoffs = [email_agent]

new_sales_manager = Agent(
    name="New_sales_manager",
    instructions=sales_manager_instructions,
    tools=new_tools_list,
    handoffs=handoffs,
    model="gpt-4o-mini"
)
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(new_sales_manager, message);

In [32]:
class NameValidationOutput(BaseModel):
    is_there_a_name: bool
    name:str
    

In [33]:
guarail_agent = Agent(
    name="Guardrail Checker",
    instructions="Check whether the input message contains a name",
    model=gemini_model,
    output_type=NameValidationOutput,
                      )


In [34]:
@input_guardrail
async def name_guardrail(ctx, agent, message):
    result = await Runner.run(guarail_agent, message, context=ctx.context )
    return GuardrailFunctionOutput(output_info=result.final_output, tripwire_triggered=result.final_output.is_there_a_name)

In [37]:
sales_manager_with_guardrails = Agent(
    name="Sales Manager with Guardrails",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    input_guardrails=[name_guardrail],
)
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace ("SDR with Guardrai"):
    result = await Runner.run(sales_manager_with_guardrails,message)
    print(results)

[RunResult(input='Write a cold email', new_items=[MessageOutputItem(agent=Agent(name='Professional_sales_agent', instructions='Your are a sales agent working for BetaLaunch.\n\nIt is a company which is oriented at providing software solutions for businesses such as web applications and mobile applications\n\nYour write in a professional and serious tone\n\n', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_685fe5b5dcb0819b93b5ca663878af0006e582290